<a href="https://colab.research.google.com/github/dasunhq/gptsniffer-implementations/blob/main/GPTSniffer_Impl_AIGCodeSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# 1. Load the dataset from Hugging Face
DATASET_NAME = "basakdemirok/AIGCodeSet"
raw_datasets = load_dataset(DATASET_NAME)

# 2. Define the pre-trained CodeBERT model
MODEL_CHECKPOINT = "microsoft/codebert-base"

# 3. Load the CodeBERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/all_data_with_ada_embeddings_will_b(…):   0%|          | 0.00/265M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7583 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7583 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [3]:
def tokenize_function(examples):
    return tokenizer(
        examples["code"],
        padding="max_length",
        truncation=True
    )

# Apply the tokenization to the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Prepare data for training: Rename 'label' column and remove unused columns
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["code"])
tokenized_datasets.set_format("torch")

# Split the dataset for training and evaluation
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

Map:   0%|          | 0/7583 [00:00<?, ? examples/s]

Map:   0%|          | 0/7583 [00:00<?, ? examples/s]

In [6]:
# 1. Load the model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=2 # Binary classification (Human vs. AI)
)

# 2. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./gptsnipper_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./gptsnipper_logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

# 3. Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,

)

# 4. Start Fine-Tuning
print("Starting GPTSniffer Fine-Tuning...")
trainer.train()
print("Fine-Tuning Complete. The resulting model is your GPTSniffer.")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting GPTSniffer Fine-Tuning...


Epoch,Training Loss,Validation Loss
1,0.574500,0.557843
2,0.532600,0.519219
3,0.436600,0.451834


Fine-Tuning Complete. The resulting model is your GPTSniffer.


In [7]:
# 1. Evaluate the final model
print("\n--- Final Evaluation ---")
metrics = trainer.evaluate()
print(metrics)

# 2. Save the final GPTSniffer model
model.save_pretrained("./gptsnipper_final_model")
tokenizer.save_pretrained("./gptsnipper_final_model")


--- Final Evaluation ---


{'eval_loss': 0.4518338143825531, 'eval_runtime': 208.1616, 'eval_samples_per_second': 36.428, 'eval_steps_per_second': 4.554, 'epoch': 3.0}


('./gptsnipper_final_model/tokenizer_config.json',
 './gptsnipper_final_model/special_tokens_map.json',
 './gptsnipper_final_model/vocab.json',
 './gptsnipper_final_model/merges.txt',
 './gptsnipper_final_model/added_tokens.json',
 './gptsnipper_final_model/tokenizer.json')

In [8]:
from transformers import pipeline

# Load the saved model and tokenizer
model_path = "./gptsnipper_final_model"
tokenizer_path = "./gptsnipper_final_model"

# Create a pipeline for sequence classification
classifier = pipeline(
    "sentiment-analysis",
    model=model_path,
    tokenizer=tokenizer_path,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


In [12]:
# Prepare some example code snippets
code_1 = """
num = int(input("Enter number:"))

if(num%2 == 0):
	print(num, " is an even number.")
else:
	print(num, " is an odd number.")
"""

code_2 = """
def check_even_odd():
    try:
        number = int(input("Enter a number: "))
        if number % 2 == 0:
            print(f"{number} is an even number.")
        else:
            print(f"{number} is an odd number.")
    except ValueError:
        print("Please enter a valid integer.")

# Run the function
if __name__ == "__main__":
    check_even_odd()
"""

# Classify the example code snippets
results_code1 = classifier(code_1)
results_code2 = classifier(code_2)

# LABEL_0 - Human-written
# LABEL_1 - AI-generated

print(results_code1)
print(results_code2)

[{'label': 'LABEL_0', 'score': 0.9048044681549072}]
[{'label': 'LABEL_1', 'score': 0.9943059086799622}]
